In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf

# Import project modules
from conllu_reader import ConlluReader
from algorithm import ArcEager, Transition, Sample
from conllu_token import Token
from model import ParserMLP

# Check for GPU availability (Optional)
print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-11-26 17:07:56.458542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-26 17:07:56.470637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-26 17:07:56.474420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-26 17:07:56.484032: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Version: 2.17.0
Num GPUs Available:  1


I0000 00:00:1764176879.297205    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176879.373859    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176879.373905    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
dataset_path = "dataset.pkl"

if not os.path.exists(dataset_path):
    print(f"Error: '{dataset_path}' not found. Please run main.py first to generate it.")
else:
    print(f"Loading dataset from {dataset_path}...")
    with open(dataset_path, "rb") as f:
        data = pickle.load(f)
        
    training_samples = data["training_samples"]
    dev_samples = data["dev_samples"]
    deprels = data["deprels"]
    actions = data["actions"]
    
    print("Dataset loaded successfully!")
    print(f"Training Samples: {len(training_samples)}")
    print(f"Development Samples: {len(dev_samples)}")
    print(f"Unique Dependency Labels: {len(deprels)}")

Loading dataset from dataset.pkl...
Dataset loaded successfully!
Training Samples: 81182
Development Samples: 4978
Unique Dependency Labels: 43


In [4]:
# Inspect the first training sample
sample = training_samples[80000]

print(f"--- Sample 1 ---")
# Note: Accessing internal state for visualization
print(f"State Stack: {[t.form for t in sample.state.S]}")
print(f"State Buffer (first 3): {[t.form for t in sample.state.B[:3]]}...")
print(f"Gold Transition: {sample.transition}")

--- Sample 1 ---
State Stack: ['ROOT', 'The']
State Buffer (first 3): ['revolution', 'in', 'the']...
Gold Transition: LEFT-ARC-det


In [5]:
# Extract features from the loaded sample
feats = sample.state_to_feats(nstack_feats=2, nbuffer_feats=2)

print("Feature List:", feats)

# Expected output format: 
# [Stack_Word_1, Stack_Word_0, Buffer_Word_0, Buffer_Word_1, 
#  Stack_UPOS_1, Stack_UPOS_0, Buffer_UPOS_0, Buffer_UPOS_1]

Feature List: ['The', 'ROOT', 'revolution', 'in', 'DET', 'ROOT_UPOS', 'NOUN', 'ADP']


In [6]:
# Initialize Model
# You can adjust dimensions and epochs as needed
model = ParserMLP(word_emb_dim=100, hidden_dim=64, epochs=3, batch_size=32)

print("Starting training...")
# The train function handles vocabulary building and vectorization internally
model.train(training_samples, dev_samples)

Starting training...
Building vocabulary...
Vocab built: 6872 words, 20 UPOS tags.
Detected Feature Shape: 4 words, 4 tags.
Vectorizing data...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 words_input (InputLayer)    [(None, 4)]                  0         []                            
                                                                                                  
 upos_input (InputLayer)     [(None, 4)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 4, 100)               687200    ['words_input[0][0]']         
                                                                                                  
 embedding_1 (Embedding)     (None, 4, 50)       

I0000 00:00:1764176933.148921    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176933.148973    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176933.148984    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176933.394981    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764176933.395019    1137 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-26

Epoch 1/3


'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring f

   3/2537 [..............................] - ETA: 4:25 - loss: 4.1958 - action_output_loss: 1.3837 - deprel_output_loss: 3.7494 - action_output_accuracy: 0.3125 - deprel_output_accuracy: 0.0521  

...

2384/2537 [===========================>..] - ETA: 3s - loss: 0.7315 - action_output_loss: 0.3150 - deprel_output_loss: 0.5554 - action_output_accuracy: 0.8851 - deprel_output_accuracy: 0.8254 

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


2537/2537 [==============================] - 21s 8ms/step - loss: 0.7310 - action_output_loss: 0.3146 - deprel_output_loss: 0.5552 - action_output_accuracy: 0.8852 - deprel_output_accuracy: 0.8254 - val_loss: 0.8385 - val_action_output_loss: 0.3999 - val_deprel_output_loss: 0.5847 - val_action_output_accuracy: 0.8614 - val_deprel_output_accuracy: 0.8252
Epoch 3/3
 678/2537 [=======>......................] - ETA: 1:49 - loss: 0.5687 - action_output_loss: 0.2322 - deprel_output_loss: 0.4487 - action_output_accuracy: 0.9160 - deprel_output_accuracy: 0.8539

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


2537/2537 [==============================] - 30s 12ms/step - loss: 0.5757 - action_output_loss: 0.2393 - deprel_output_loss: 0.4485 - action_output_accuracy: 0.9135 - deprel_output_accuracy: 0.8538 - val_loss: 0.8854 - val_action_output_loss: 0.4333 - val_deprel_output_loss: 0.6028 - val_action_output_accuracy: 0.8606 - val_deprel_output_accuracy: 0.8242


In [10]:
# 1. Load a sentence for testing (e.g., from the dev set)
reader = ConlluReader()
dev_trees = reader.read_conllu_file("en_partut-ud-dev_clean.conllu", inference=False)
dev_trees = reader.remove_non_projective_trees(dev_trees)

# Pick a sentence
original_tree = dev_trees[5]
input_tokens = []

# Create a clean version of the tokens (simulating inference mode)
for t in original_tree:
    new_token = Token(t.id, t.form, t.lemma, t.upos, t.cpos, t.feats, "_", "_") 
    input_tokens.append(new_token)

# 2. Run the model
print("Running inference on sentence:")
print(" ".join([t.form for t in input_tokens if t.id != 0]))

# Pass as a list of sentences (even if just one)
parsed_sents = model.run([input_tokens])
parsed_tree = parsed_sents[0]

# 3. Inspect Results
print("\nParsed Result:")
print(f"{'ID':<3} {'FORM':<15} {'HEAD':<5} {'DEPREL'}")
print("-" * 40)
for t in parsed_tree:
    if t.id == 0: continue
    print(f"{t.id:<3} {t.form:<15} {t.head:<5} {t.dep}")

# 4. Compare with Gold Standard
correct_heads = 0
total_tokens = 0
for pred, gold in zip(parsed_tree, original_tree):
    if pred.id == 0: continue
    total_tokens += 1
    if str(pred.head) == str(gold.head):
        correct_heads += 1

uas = correct_heads / total_tokens if total_tokens > 0 else 0
print(f"\nUnlabeled Attachment Score (UAS): {uas:.2f}")

Running inference on sentence:
You have requested a debate on this subject in the course of the next few days , during this part - session .

Parsed Result:
ID  FORM            HEAD  DEPREL
----------------------------------------
1   You             3     nsubj
2   have            3     aux
3   requested       0     root
4   a               5     det
5   debate          3     obj
6   on              8     case
7   this            8     det
8   subject         3     obj
9   in              11    case
10  the             11    det
11  course          8     nmod
12  of              16    case
13  the             16    det
14  next            16    amod
15  few             16    amod
16  days            11    nmod
17  ,               8     acl
18  during          22    case
19  this            22    det
20  part            22    amod
21  -               22    punct
22  session         8     nmod
23  .               3     punct

Unlabeled Attachment Score (UAS): 0.83


In [11]:
from conllu_reader import ConlluReader

# Initialize reader if not already done
reader = ConlluReader()

# Load the test set
# We use inference=True to ensure we treat this as unparsed data
print("Loading test set...")
test_trees = reader.read_conllu_file("en_partut-ud-test_clean.conllu", inference=True)
print(f"Loaded {len(test_trees)} sentences for testing.")

Loading test set...
Loaded 153 sentences for testing.


In [17]:
test_trees[1]

[0	ROOT	ROOT	ROOT_UPOS	ROOT_CPOS	ROOT_FEATS	_	_	_	_,
 1	Any	any	DET	DI	PronType=Ind	2	det	_	_,
 2	use	use	NOUN	S	Number=Sing	0	root	_	_,
 3	of	of	ADP	E	_	5	case	_	_,
 4	the	the	DET	RD	Definite=Def|PronType=Art	5	det	_	_,
 5	work	work	NOUN	S	Number=Sing	2	nmod	_	_,
 6	other	other	ADJ	A	Degree=Pos	2	nsubj	_	_,
 7	than	than	SCONJ	CS	_	9	mark	_	_,
 8	as	as	ADP	E	_	9	mark	_	_,
 9	authorized	authorize	VERB	V	Tense=Past|VerbForm=Part	6	acl	_	_,
 10	under	under	ADP	E	_	12	case	_	_,
 11	this	this	DET	DD	Number=Sing|PronType=Dem	12	det	_	_,
 12	license	license	NOUN	S	Number=Sing	9	obj	_	_,
 13	or	or	CCONJ	CC	_	14	cc	_	_,
 14	copyright	copyright	NOUN	S	Number=Sing	12	nmod	_	_,
 15	law	law	NOUN	S	Number=Sing	14	nmod	_	_,
 16	is	be	AUX	VA	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	17	aux	_	_,
 17	prohibited	prohibit	VERB	V	Tense=Past|VerbForm=Part	9	advcl	_	_,
 18	.	.	PUNCT	FS	_	2	punct	_	_]

In [12]:
# Conduct inference on the test set
print("Running inference on the test set...")
# The model.run method modifies the trees in-place or returns them
# It predicts the HEAD and DEPREL for each token
predicted_test_trees = model.run(test_trees)

print("Inference complete.")

Running inference on the test set...
Inference complete.


In [18]:
# Define the output path for the raw predictions
output_path = "output_test_raw.conllu"

# Write the predicted trees to the file
print(f"Saving raw predictions to {output_path}...")
reader.write_conllu_file(output_path, predicted_test_trees)
print("File saved.")

Saving raw predictions to output_test_raw.conllu...
File saved.


In [19]:
from postprocessor import PostProcessor

# Initialize the postprocessor
post = PostProcessor()

print(f"Post-processing predictions in {output_path}...")

# The postprocess method reads the file, fixes issues, and returns the list of fixed trees
fixed_trees = post.postprocess(output_path)

# Save the fixed trees to a new file (this is the one you should evaluate)
final_output_path = "output_test_fixed.conllu"
reader.write_conllu_file(final_output_path, fixed_trees)

print(f"Post-processing complete. Final predictions saved to {final_output_path}")

Post-processing predictions in output_test_raw.conllu...
Post-processing complete. Final predictions saved to output_test_fixed.conllu


In [20]:
# Run the evaluation script comparing the Gold Standard (test_clean) against your Fixed Output
# -v provides verbose output
!python conll18_ud_eval.py en_partut-ud-test_clean.conllu output_test_fixed.conllu -v

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |    100.00 |    100.00 |    100.00 |    100.00
UAS        |     74.91 |     74.91 |     74.91 |     74.91
LAS        |     63.50 |     63.50 |     63.50 |     63.50
CLAS       |     47.96 |     46.94 |     47.45 |     46.94
MLAS       |     46.47 |     45.48 |     45.97 |     45.48
BLEX       |     47.96 |     46.94 |     47.45 |     46.94
